# Exploratory Data Analysis

In [ ]:
import json
import math
import pickle
import re
import sys
from collections import Counter, defaultdict
from functools import reduce
from pathlib import Path

import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import toolz
from rich import print
from tqdm.auto import tqdm

sys.path.insert(0, "../src/")

tqdm.pandas()
# plt.style.use("ggplot")
sns.set()

from dataset import (
    TagAssociations,
    filter_by_kinds,
    filter_by_thr,
    get_tag_ranking,
    normalise_wrt,
)
from parse_avclass import parse_avclass_results

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

## Vendors

Parse VT results (input is a `.jsonl` file, as required by AVClass)/

In [ ]:
vendors = defaultdict(list)

with open("../data/vt_results.jsonl", "rt") as fp:
    for line in tqdm(fp):
        x = json.loads(line)
        for v in x["attributes"]["last_analysis_results"]:
            vendors[v].append(x["attributes"]["sha256"])

In [ ]:
len(vendors)

In [ ]:
Counter(toolz.dicttoolz.valmap(len, vendors)).most_common()

## Load original EMBER metadata

In [ ]:
ember = pd.read_csv("../data/raw/ember_original_metadata.csv", index_col=0)
ember.info()

Check how many hashes were unprocessed (no VT results)

In [ ]:
ext_vt = set()

with open(VT_JSONL) as fp:
    for line in tqdm(fp):
        j = json.loads(line)
        ext_vt.add(j["attributes"]["sha256"])

proc = ember.query("sha256.isin(@ext_vt)")

In [ ]:
pd.crosstab(proc["label"], [proc["subset"]], margins=True)

### Label by date appeared

In [ ]:
colors = ["gray", "skyblue", "salmon"]

In [ ]:
thr = 2018
ember["date"] = ember["appeared"].apply(
    lambda x: x if int(x.split("-")[0]) >= thr else f"<{thr}"
)
x = (
    ember.groupby("date")["label"]
    .apply(lambda xs: xs.value_counts().to_dict())
    .fillna(0)
    .to_frame(name="labels")
    .reset_index()
    .set_index("date")
    .pivot(columns=["level_1"], values=["labels"])
)
x = x.iloc[[-1, *range(0, len(x.index) - 1)]]
x.columns = ["unlabelled", "benign", "malicious"]
fig, ax = plt.subplots(figsize=(8, 4))
x.plot(kind="bar", stacked=True, color=colors, ax=ax)
ix = np.linspace(0, 120_000, num=13)
ax.set_yticks(ix, labels=[f"{int(i/1000)}k" for i in ix])
ax.set_xlabel("Date appeared")
ax.set_ylabel("Number of samples")
pass

In [ ]:
x = (
    ember.groupby("subset")["label"]
    .apply(lambda xs: xs.value_counts().to_dict())
    .fillna(0)
    .to_frame(name="labels")
    .reset_index()
    .set_index("subset")
    .pivot(columns=["level_1"], values=["labels"])
)
x.columns = ["unlabelled", "benign", "malicious"]
fig, ax = plt.subplots(figsize=(3, 4))
x.plot(kind="bar", stacked=True, color=colors, ax=ax)
ix = np.linspace(0, 800_000, num=9)
ax.set_yticks(ix, labels=[f"{int(i/1000)}k" for i in ix])
ax.set_xlabel("Subset")
ax.set_ylabel("Number of samples")
pass

## Interpreting AVClass results

In [ ]:
avclass_results = parse_avclass_results("../data/avclass_results.txt")

renamer = {
    "avclass": "avclass_prev",  # previous avclass results (EMBER, 2018)
    "tags": "avclass_curr",  # current avclass results
}

# augmented dataframe: prev ember | curr avclass results
aug_df = ember.merge(avclass_results, how="left", on="sha256").rename(renamer, axis=1)

# both: prev ember & curr avclass
both_df = ember.merge(avclass_results, how="inner", on="sha256").rename(renamer, axis=1)

### Tag presence vs label

In [ ]:
mask_curr = aug_df["avclass_curr"].apply(
    lambda d: pd.notna(d)
    and ("FAM:" in str(d) or ("CLASS:" in str(d) and "CLASS:grayware" not in str(d)))
)
mask_prev = aug_df["avclass_prev"].notna()
pd.crosstab(aug_df["label"], [mask_prev, mask_curr], margins=True)

### Load associations (tag co-occurrence)
i.e. tag pairs co-occurring, taken from the `.alias` file generated by AVClass

For example, the following line:
```
t1             t2          |t1|  |t2|      |t1^t2|  |t1^t2|/|t1|    |t1^t2|/|t2|
FAM:lockergoga FAM:fareit   3    22170     1        0.33            0.00
```
means that:
- `FAM:lockergoga` appears in 3 samples (total)
- `FAM:fareit` appears in 22,170 samples (total)
- they appear together in 1 sample

We can use `CLASS` tag to obtain a higher-level organization of samples.
**NOTE:** the CLASSes are not disjoint, i.e. a sample can be part of multiple CLASSes.

This is useful to enrich tag info for a sample with info from different samples.

In [ ]:
def most_common(xs: dict) -> list[tuple]:
    return sorted(xs.items(), key=lambda kv: kv[1], reverse=True)

In [ ]:
assoc = TagAssociations(file="../data/avclass_tag_co_occurrence.alias")

In [ ]:
len(filter_by_kinds(assoc.pair_norm_freq, kinds=["CLASS"]).keys())

In [ ]:
# what other families co-occur with `FAM:lockergoga`?
print(filter_by_kinds(assoc.pair_counter["FAM:lockergoga"], kinds=["FAM"]))

In [ ]:
# what other classes co-occur with `FAM:lockergoga`?
print(
    most_common(filter_by_kinds(assoc.pair_counter["FAM:lockergoga"], kinds=["CLASS"]))
)

In [ ]:
# what are the families and classes which co-occur with `FAM:cosmu` ≥ 80% of the times, w.r.t. the freq. of `FAM:cosmu`?
filter_by_thr(
    filter_by_kinds(assoc.pair_norm_freq["FAM:cosmu"], kinds=["FAM", "CLASS"]), thr=0.8
)

### Top-N most frequent tags by kind

In [ ]:
def plot_kind(kind, N, ax):
    tmp = Counter()
    for t, c in most_common(filter_by_kinds(assoc.tag_counter, kinds=[kind])):
        tmp[t.split(":")[1]] += c
    tmp = pd.DataFrame(tmp.most_common()[:N], columns=["name", "count"])

    tmp = tmp.sort_values(by="count")
    tmp.plot(kind="barh", x="name", y="count", ax=ax, color="salmon")
    ax.set_yticklabels(tmp["name"], fontfamily="monospace", fontsize=8)
    ax.set_xscale("log")
    ax.grid(True, axis="both", which="both")
    ax.set_xlabel("Frequency")
    ax.set_ylabel("")
    ax.get_legend().remove()
    ax.set_title(f"{kind} tags")


N = 20
plt.figure(figsize=(16, 8))
gs = gridspec.GridSpec(2, 4)
gs.update(hspace=0.3, wspace=1)
plot_kind("CLASS", N, ax=plt.subplot(gs[0, :2]))
plot_kind("FAM", N, ax=plt.subplot(gs[0, 2:]))
plot_kind("BEH", N, ax=plt.subplot(gs[1, 1:3]))
plt.suptitle(f"Top {N} most frequent tags by kind")
plt.show()
pass

### VT detections

In [ ]:
tmp = aug_df.query("vt_detections > 0")[["label", "vt_detections"]]

figsize = (8, 4)

plt.figure(figsize=figsize)
sns.histplot(
    tmp,
    x="vt_detections",
    hue="label",
    stat="percent",
    bins=32,
    palette={0: "skyblue", 1: "salmon", -1: "gray"},
    alpha=0.9,
)
plt.title("Detections per label")
pass

In [ ]:
def agg(xs):
    ys = Counter(xs)
    total = sum(ys.values())
    return {k: v / total for k, v in ys.items()}


bin_ranges = list(range(0, aug_df["vt_detections"].max(), 5))
x = (
    tmp.groupby(pd.cut(tmp["vt_detections"], bins=bin_ranges))["label"]
    .apply(agg)
    .fillna(0)
    .reset_index()
    .set_index("vt_detections")
    .pivot(columns="level_1", values="label")
)
x.columns.name = "label"
fig, ax = plt.subplots(figsize=(8, 4))
x.plot(kind="bar", stacked=True, color=colors, alpha=0.8, ax=ax)
plt.legend(loc="upper right", bbox_to_anchor=(1.12, 1))
pass

### Potential FPs & FNs

In [ ]:
def get_tags_by_prefix(prefix: str, tags: dict) -> dict:
    return {k: v for k, v in tags.items() if k.startswith(prefix)}

In [ ]:
fps = both_df.query("(label == 1) & (vt_detections == 0)")
print("Potential FPs:", fps["subset"].value_counts())

In [ ]:
family_exists = both_df.query("avclass_curr.notna()")["avclass_curr"].apply(
    lambda tags: len(get_tags_by_prefix("FAM", tags)) > 0
)
fns = both_df.query("(label == 0) & (vt_detections > 0) & (@family_exists)")
print("Potential FNs:", fns["subset"].value_counts())

### Agreement b/w prev & curr AVClass results

In [ ]:
# data subset for which both prev & curr AVClass results exist
common_df = both_df.query("(avclass_prev.notna()) & (avclass_curr.notna())")
common_df.shape

In [ ]:
def agreement(prev: str, curr: dict) -> bool:
    """Example
    prev = zbot
    curr = {'FAM:zbot': 29, 'FILE:os:windows': 16), 'CLASS:spyware': 4]
    """
    for kind, _ in curr.items():
        if prev.lower() in kind.lower():
            return True

    return False

In [ ]:
agree_mask = common_df.apply(
    lambda row: agreement(prev=row["avclass_prev"], curr=row["avclass_curr"]), axis=1
)
print(agree_mask.value_counts())

In [ ]:
print("Agreement")
tmp = common_df[agree_mask]
print(pd.crosstab(tmp["label"], [tmp["subset"]], margins=True))

print("Disagreement")
tmp = common_df[~agree_mask]
print(pd.crosstab(tmp["label"], [tmp["subset"]], margins=True))

### High-level mapping: `CLASS` tag

In [ ]:
tmp = filter_by_kinds(assoc.tag_counter, kinds=["CLASS"])
print(f"CLASSes (most common first), total={len(tmp)}")
print(most_common(tmp))

In [ ]:
clz, xs = map(np.array, zip(*most_common(tmp)))
clz = [x[6:] for x in clz]
plt.figure(figsize=(8, 6))
ax = sns.barplot(x=xs, y=clz, color="salmon")
ax.set_yticklabels(clz, fontfamily="monospace", fontsize=10)
plt.tight_layout()
ix = np.linspace(0, 6, num=13)
ax.set_xscale("log")
plt.title("CLASS tag frequency")
plt.xlabel("Number of samples")
pass

In [ ]:
high_level_classes = Counter()
for c, num in tmp.items():
    high_level_classes[c.split(":")[1]] += num

print(f"High-level CLASSes (most common first), total={len(high_level_classes)}")
print(high_level_classes.most_common())

### `CLASS` to `FAM`/`UNK` mapping

In [ ]:
class2fam = defaultdict(dict)

for c, rest in filter_by_kinds(assoc.pair_norm_freq, kinds=["CLASS"]).items():
    for f, freq in filter_by_kinds(rest, kinds=["FAM", "UNK"]).items():
        class2fam[c][f] = freq
        assert math.isclose(
            freq,
            assoc.pair_counter[c][f] / assoc.tag_counter[c],
            rel_tol=0,
            abs_tol=1e-8,
        )

class2fam = dict(class2fam)

In [ ]:
Counter(filter_by_kinds(class2fam["CLASS:worm"], kinds=["FAM"])).most_common()[:10]

---

## Examples

`prev` and `curr` agree

`prev` and `curr` disagree